<a href="https://colab.research.google.com/github/essiesalari/Australian-Celebrities-Face-Recognition-VGG16/blob/main/Australian_Celebrities_Face_Recognition_VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Australian Celebrities Face Recognition-VGG16**

## Import Libraries

In [4]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras

## Import datatset

In [5]:
! pip install bing_image_downloader

In [6]:
from bing_image_downloader import downloader

In [7]:
# Query
search_queries=[
    'Nicole Kidman',
    'Mel Gibson',
    'Liam Hemsworth',
    'Cathy Freeman',
    'Margot Robbie'
]

dataset_path = 'dataset'

# Imgae downloder function
def donwloader(query, n_sample, dataset_name):
  downloader.download(query, limit=n_sample, output_dir=os.path.join(dataset_path, dataset_name))

for query in search_queries:
  donwloader(query, n_sample=20, dataset_name='train')
  donwloader(query, n_sample=10, dataset_name='val')
  donwloader(query, n_sample=3, dataset_name='test')

[%] Downloading Images to /content/dataset/train/Nicole Kidman


[!!]Indexing page: 1

[%] Indexed 20 Images on Page 1.


[%] Downloading Image #1 from https://celebmafia.com/wp-content/uploads/2016/04/nicole-kidman-the-family-fang-premiere-in-new-york-city-3.jpg
[%] File Downloaded !

[%] Downloading Image #2 from http://networthcelebrities.com/wp-content/uploads/2016/02/Nicole-Kidman-_11.jpg
[%] File Downloaded !

[%] Downloading Image #3 from https://celebmafia.com/wp-content/uploads/2018/01/nicole-kidman-2018-critics-choice-awards-10.jpg
[%] File Downloaded !

[%] Downloading Image #4 from http://celebmafia.com/wp-content/uploads/2016/05/nicole-kidman-2016-met-gala-held-at-the-metropolitan-museum-of-art-new-york-1.jpg
[%] File Downloaded !

[%] Downloading Image #5 from https://celebmafia.com/wp-content/uploads/2016/04/nicole-kidman-academy-of-country-music-awards-2016-in-las-vegas-1.jpg
[%] File Downloaded !

[%] Downloading Image #6 from http://images2.fanpop.com/images/photos/41

In [8]:
dataset_path='/content/dataset'
train_path = os.path.join(dataset_path, 'train')
val_path = os.path.join(dataset_path, 'val')
test_path = os.path.join(dataset_path, 'test')

In [9]:
nr_files = nr_train_files = nr_val_files = nr_test_files = 0

for root, dirc, files in os.walk(dataset_path):
  nr_files += len(files)
print("#files: ", nr_files)
for root, dirc, files in os.walk(train_path):
  nr_train_files += len(files)
print("#train_files: ", nr_train_files)
for root, dirc, files in os.walk(val_path):
  nr_val_files += len(files)
print("#val_files: ", nr_val_files)
for root, dirc, files in os.walk(test_path):
  nr_test_files += len(files)
print("#test_files: ", nr_test_files)

#files:  140
#train_files:  100
#val_files:  28
#test_files:  12


## Build  a New Model based on VGG16

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [11]:
batch_size = 128
lr = 0.001
epoch = 100

# Define Image data generator
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen =  ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)


Found 100 images belonging to 5 classes.
Found 28 images belonging to 5 classes.
Found 12 images belonging to 5 classes.


In [ ]:
# Load VGG16
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add global average layer
net = base_model.output
net = GlobalAveragePooling2D()(net)

# Creat the new model with VGG16 (feature selection) + SVC (classifier)
new_model = Model(inputs=base_model.input, outputs=net)
#new_model.compile(optimizer=Adam(learning_rate=lr), loss='categorical_crossentropy', metrics=['accuracy'])

train_features = new_model.predict(train_generator)
train_labels = np.argmax(train_generator.labels, axis=1)

val_features = new_model.predict(val_generator)
val_labels = np.argmax(val_generator.labels, axis=1)

# Train SVM Classifier
svm = SVC(kernel='linear')
svm.fit(train_features, train_labels )

## Train and Evaluate the New_Model

In [ ]:
# Evaluate on validation set
val_accuracy = svm.score(val_features, val_labels)
print("Validation Accuracy:", val_accuracy)